----------

## <span style="color:green">Комментарий от наставника<span>

Привет! Ты проделал большую работу, молодец! Пришлось скачать и выполнять локально.
    
Круто сделал! `class_weight='balanced'` - отлично
    
### Что можно сделать лучше:

1. Посчитать константную модель и проверить на адекватность
2. Для улучшения модели можно поиграть с параметрами `TfidfVectorizer`, например `min_df`

По твоим вопросам:

Логика решения правильная.
На бустинге падает ядро - думаю, это проблема ЯНдекса, мощностей не хватает, много студентов запускают свои работы.
Для оптимазции кода, где создаётся ДатаФрейм используй `apply` например: `df['text_lemma'] = df['text'].apply(get_lemm)`

---------

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
#rom pymystem3 import Mystem
import re
from nltk.stem import SnowballStemmer
# импортируйте класс Mystem из библиотеки pymystem3

In [2]:
df_toxic_comments = pd.read_csv('C:\\Users\\Dima\\Downloads\\toxic_comments.csv')

In [3]:
#df_toxic_comments = df_toxic_comments.iloc[:30000]

In [4]:
df_toxic_comments

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [5]:
corpus = df_toxic_comments['text'].values.astype('U')

In [6]:
print(df_toxic_comments.head)

<bound method NDFrame.head of                                                      text  toxic
0       Explanation\nWhy the edits made under my usern...      0
1       D'aww! He matches this background colour I'm s...      0
2       Hey man, I'm really not trying to edit war. It...      0
3       "\nMore\nI can't make any real suggestions on ...      0
4       You, sir, are my hero. Any chance you remember...      0
...                                                   ...    ...
159566  ":::::And for the second time of asking, when ...      0
159567  You should be ashamed of yourself \n\nThat is ...      0
159568  Spitzer \n\nUmm, theres no actual article for ...      0
159569  And it looks like it was actually you who put ...      0
159570  "\nAnd ... I really don't think you understand...      0

[159571 rows x 2 columns]>


In [7]:
class_frequency = df_toxic_comments['toxic'].value_counts(normalize=True)
print(class_frequency)
class_frequency.plot(kind='bar')

0    0.898321
1    0.101679
Name: toxic, dtype: float64


<span style="color:blue">видим сильный дисбаланс классов, но как с этим бороться?? если делать UPSAMPLE, то еще больше увеличивать DataFrame, и так ядро падает, на бустинге , например. Сделал в LogisticRegression 'class_weight = 'balanced''.

In [8]:
print(corpus)

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
 ...
 'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.'
 'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'
 '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead o

In [9]:
# english_stemmer = SnowballStemmer('english')
# def stemmed(text):
#     stem_sentence=[]
#     for word in text.split():
#         stem_sentence.append(english_stemmer.stem(word))
#         stem_sentence.append(" ")
#     return "".join(stem_sentence)

In [10]:
# делаю стемминг
english_stemmer = SnowballStemmer('english')
def stemmed(text):
   # st=[english_stemmer.stem(word) for word in text.split()]
    return " ".join([english_stemmer.stem(word) for word in text.split()])

In [11]:
# оставляю диапазон букв
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = " ".join(text.split())
    return text

print("Исходный текст:", corpus[0])
print("Очищенный и лемматизированный текст:", stemmed(clear_text(corpus[0])))

Исходный текст: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Очищенный и лемматизированный текст: explan whi the edit made under my usernam hardcor metallica fan were revert they weren t vandal just closur on some gas after i vote at new york doll fac and pleas don t remov the templat from the talk page sinc i m retir now


In [12]:
len(df_toxic_comments)

159571

In [13]:
#делаю фрейм 
df = pd.DataFrame(columns=['stem_text'])
for i in range(len(df_toxic_comments)):
    df = df.append({'stem_text': stemmed(clear_text(corpus[i]))}, ignore_index=True)

<span style="color:blue">очень долго создает этот Фрейм, может по другому как-то можно сделать?

In [14]:
print(df)

                                                stem_text
0       explan whi the edit made under my usernam hard...
1       d aww he match this background colour i m seem...
2       hey man i m realli not tri to edit war it s ju...
3       more i can t make ani real suggest on improv i...
4       you sir are my hero ani chanc you rememb what ...
...                                                   ...
159566  and for the second time of ask when your view ...
159567  you should be asham of yourself that is a horr...
159568  spitzer umm there no actual articl for prostit...
159569  and it look like it was actual you who put on ...
159570  and i realli don t think you understand i came...

[159571 rows x 1 columns]


In [15]:
df_toxic =  pd.concat([df_toxic_comments,df], axis=1)

In [16]:
#добавляю в исходный
df_toxic

,text,toxic,stem_text
0,Explanation\nWhy the edits made under my usern...,0,explan whi the edit made under my usernam hard...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m realli not tri to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make ani real suggest on improv i...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero ani chanc you rememb what ...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,and for the second time of ask when your view ...
159567,You should be ashamed of yourself \n\nThat is ...,0,you should be asham of yourself that is a horr...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm there no actual articl for prostit...
159569,And it looks like it was actually you who put ...,0,and it look like it was actual you who put on ...


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [18]:
#разбиваю на train и test
train_features, test_features, train_target, test_target = train_test_split(df_toxic['stem_text'], 
                                                                            df_toxic['toxic'],  test_size=0.2)

In [19]:
test_target

82321     0
139196    0
24228     0
124633    0
136792    1
         ..
136115    1
85816     0
92902     0
8065      0
13061     0
Name: toxic, Length: 31915, dtype: int64

In [20]:
#corpus_train = train_features.values.astype('U')

In [21]:
#corpus_test = test_features.values.astype('U')

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [23]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dima\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
stop_words = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

In [25]:
#посчитаю TF-IDF для корпуса текстов
tf_idf_train = count_tf_idf.fit_transform(train_features)

In [26]:
tf_idf_train.shape

(127656, 115198)

In [27]:
#print("Размер:", n_gramm.shape)

In [28]:
tf_idf_test = count_tf_idf.transform(test_features)

In [29]:
model = LogisticRegression(class_weight = 'balanced')
model.fit(tf_idf_train, train_target)

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
predict = model.predict(tf_idf_test)

In [31]:
predict.shape

(31915,)

In [32]:
from sklearn.metrics import f1_score

In [33]:
print(f1_score(test_target, predict))

0.742505133470226


<span style="color:blue">получили f1 < 0.75

In [34]:
from catboost import CatBoostClassifier

In [35]:
model= CatBoostClassifier(iterations=1000,
                               random_seed = 42,
                               metric_period = 50,
                               eval_metric = 'F1',
                               learning_rate = 0.5,
                               early_stopping_rounds = 50, 
                               #task_type="GPU", 
                               #devices='0:1'
                              )

<span style="color:blue">долго делает локально, ядро падает на практикуме 

In [36]:

model.fit(tf_idf_train, train_target, verbose = 100)

0:	learn: 0.4678946	total: 8.1s	remaining: 2h 14m 49s
100:	learn: 0.7854351	total: 3m 40s	remaining: 32m 45s
200:	learn: 0.8235093	total: 6m 59s	remaining: 27m 46s
300:	learn: 0.8355302	total: 10m 17s	remaining: 23m 54s
400:	learn: 0.8403511	total: 13m 37s	remaining: 20m 20s
500:	learn: 0.8480783	total: 16m 55s	remaining: 16m 51s
600:	learn: 0.8543330	total: 20m 12s	remaining: 13m 25s
700:	learn: 0.8613181	total: 23m 30s	remaining: 10m 1s
800:	learn: 0.8694746	total: 26m 49s	remaining: 6m 39s
900:	learn: 0.8784203	total: 30m 12s	remaining: 3m 19s
999:	learn: 0.8973855	total: 33m 31s	remaining: 0us


In [39]:
predict = model.predict(tf_idf_test)

In [40]:
print(f1_score(test_target, predict))

0.7628318584070796


<span style="color:blue">получили f1 > 0.75

<span style="color:blue">Наставник!) Подскажи плиз, что можно улучшить , что доделать.  
    Как оптимизировать код (а то долго очень создает Фрейм со строками). На бустинге падает ядро.  
    Вообще по логике решения - правильно или нет? 

In [37]:
# count_vect = CountVectorizer()

# bow = count_vect.fit_transform(corpus)

# print("Размер мешка без учёта стоп-слов:", bow.shape)

In [38]:
# count_vect = CountVectorizer(ngram_range=(2, 2))
# n_gramm = count_vect.fit_transform(corpus)
# # создайте n-грамму n_gramm, для которой n=2
# print("Размер:", n_gramm.shape)

# 2. Обучение

# 3. Выводы

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны